# This project shows a replication from AWS RDS to Google Cloud SQL. 

In this project we create a Master-Slave design, where RDS is the Master and GCP MySQL is the Slave. 
To setup a replication on Google Cloud SQL we need to use a static IP address on a compute engine to forward the traffic from master to slave.

![](Replicaproject.png)

We are going to assume there's an RDS instance running as our Master database.
We need to modify this RDS intance and add a parameter group that has GTID enabled. See images below

![](parameter-group-edit.png)

Now add this new parameter group to the RDS instance.

![](parameter-group-add.png)

After applying these changes, RDS requires a reboot for this change to be made.
So go ahead and give it a reboot.

![](Reboot-RDS.png)

Now that's all we need to do for our Master database.

Next step is to create an EC2 intance with an elastic IP address which will act as a proxy for our Master Database. By default EC2 doesn't not have a static IP address, so creating an elastic IP address helps this proxy process(Master to Slave)

For quick automation, we use a Terraform script "ec2-build.tf"


Now SSH into the EC2 and run this socat command:
    
    socat -v TCP-LISTEN:3306,fork TCP:"YOUR-RDS-INSTANCE-ID":3306
                
See image below

![](socat.PNG)

We use SOCAT here because we want our EC2 intance to ALWAYS listen and point to our RDS Master on port 3306

With this single line command we have made our EC2 instance act as a proxy between our RDS Master and GCP SQL Slave.
You can verify this connection by using a SQL workbench tool such as 'MySQL Workbench'

See image below

![](mysql-workbench.png)

Now that's all we need to do for our Proxy EC2 instance.

Next step is to create a bucket for our SQL dumpfile of our Master database.
Once the bucket is created we can now go to a bash terminal and run this code:
    
     mysqldump \
 -h34.203.124.227 -P -urdsmaster -p \
 - databases aws-rds-master \
 - hex-blob - skip-triggers - master-data=1 \
 - order-by-primary - compact - no-autocommit \
 - default-character-set=utf8mb4 - ignore-table \
 - single-transaction - set-gtid-purged=on | gzip | gsutil cp - gs://rdsmaster/rds-master-george/rds-master.sql.gz

            

This code will get a SQL dumpfile by going to our RDS Master through our proxy EC2 intance, remember we still have our
socat command running and we are using our static IP address here to point to the proxy. This dumpfile will be sent to our bucket as a zip file.

Next, we now need to create our Slave database. Simply go to GCP SQL and begin a Migrate data process

An important thing to note here is that GCP requires a static IP address to create a replication process, which is why we used a EC2 intance with elastic IP attached as a proxy

Simply add that static IP address in the information steps and create a username/password, also choose a Database version

![](gcp-mysql.png)

Lastly, we add the SQL dumpfile from our bucket and click finish. GCP takes care of the rest :) 

![](sql-dump-1.png)

![](sql-dump-2.png)

Automatically the replication should start.
You can verify this connection by using a SQL workbench tool such as 'MySQL Workbench' and running this code:

    show slave status; #This will show the status of your Read_Master_log, slave_IO_running and slave_SQL_running
    
    
Remember to grab the Public IP address of the GCP SQL replica instance. 